In [1]:
import mvtec
import train
import vanila_ae as ae
from config import Config, print_config

In [7]:
config = Config(
    data_dir='/mnt/d/datasets/mvtec',
    batch_size=32,
    num_epochs=10,
    experiment_name='test_mvtec_vanila_ae',
)
print_config(config)

Configuration:
  data_dir: /mnt/d/datasets/mvtec
  seed: 42
  device: cuda
  img_size: 256
  normalize: False
  category: bottle
  batch_size: 32
  valid_ratio: 0.2
  latent_dim: 512
  num_epochs: 10
  learning_rate: 0.001
  weight_decay: 1e-05
  save_models: True
  save_results: True
  experiment_name: test_mvtec_vanila_ae


In [3]:
train.set_device(seed=config.seed)

train_transform, test_transform = mvtec.get_transforms(
    img_size=config.img_size,
    normalize=config.normalize
)

train_loader, valid_loader, test_loader = mvtec.get_dataloaders(
    data_dir=config.data_dir, 
    category=config.category, 
    batch_size=config.batch_size,
    train_transform=train_transform, 
    test_transform=test_transform
)

>> Random Seed: 42
>> Device: cuda
>> CUDA available: True
>> GPU: NVIDIA GeForce GTX 1080 Ti
Loaded bottle train: 209 images
>> Normal:  209
>> Anomaly: 0
Loaded bottle test: 83 images
>> Normal:  20
>> Anomaly: 63
Loaded bottle train: 209 images
>> Normal:  209
>> Anomaly: 0


In [4]:
model = ae.VanillaAutoEncoder(latent_dim=config.latent_dim).to(config.device)

total_params = sum(p.numel() for p in model.parameters())
model_size_mb = total_params * 4 / (1024 * 1024)  # float32 기준

print(f"Model parameters: {total_params:,}")
print(f"Model size: {model_size_mb:.1f} MB")

Model parameters: 30,979,715
Model size: 118.2 MB


In [5]:
trained_model, history = train.train_model(
                model, 
                train_loader, 
                valid_loader, 
                num_epochs=config.num_epochs
)

Starting training...


✓ New best model saved (Valid Loss: 0.4532)
Epoch  1/10 | Train: Loss=0.3391, Psnr=8.8885, Ssim=0.1758 | Valid: Loss=0.4532, Psnr=5.8848, Ssim=0.0911 | LR=0.001000 | Time: 6.5s


✓ New best model saved (Valid Loss: 0.3194)
Epoch  2/10 | Train: Loss=0.2635, Psnr=9.5313, Ssim=0.3817 | Valid: Loss=0.3194, Psnr=7.8083, Ssim=0.3218 | LR=0.001000 | Time: 6.4s


Epoch  3/10 | Train: Loss=0.2545, Psnr=9.5641, Ssim=0.4097 | Valid: Loss=0.4430, Psnr=5.5044, Ssim=0.1802 | LR=0.001000 | Time: 3.6s


Epoch  4/10 | Train: Loss=0.2505, Psnr=9.6181, Ssim=0.4198 | Valid: Loss=0.4242, Psnr=5.7431, Ssim=0.2077 | LR=0.001000 | Time: 6.3s


...

✓ New best model saved (Valid Loss: 0.3150)
Epoch  6/10 | Train: Loss=0.2426, Psnr=9.8471, Ssim=0.4329 | Valid: Loss=0.3150, Psnr=7.6840, Ssim=0.3478 | LR=0.001000 | Time: 6.1s


✓ New best model saved (Valid Loss: 0.3075)
Epoch  7/10 | Train: Loss=0.2224, Psnr=10.5451, Ssim=0.4647 | Valid: Loss=0.3075, Psnr=7.8667, Ssim=0.3563 | LR=0.001000 | Time: 6.1s


✓ New best model saved (Valid Loss: 0.2242)
Epoch  8/10 | Train: Loss=0.2094, Psnr=10.8107, Ssim=0.4957 | Valid: Loss=0.2242, Psnr=10.2665, Ssim=0.4722 | LR=0.001000 | Time: 3.7s


Epoch  9/10 | Train: Loss=0.2008, Psnr=10.9932, Ssim=0.5162 | Valid: Loss=0.2347, Psnr=10.5211, Ssim=0.4246 | LR=0.001000 | Time: 6.4s


✓ New best model saved (Valid Loss: 0.2079)
Epoch 10/10 | Train: Loss=0.1969, Psnr=11.1012, Ssim=0.5248 | Valid: Loss=0.2079, Psnr=10.7347, Ssim=0.5040 | LR=0.001000 | Time: 6.2s
Training completed. Best validation loss: 0.2079


In [6]:
results, detailed_results = train.evaluate_model(trained_model, test_loader)

results

Evaluating on test set...


Testing: 100%|██████████| 3/3 [00:02<00:00,  1.35it/s]


{'AUROC': np.float64(0.7761904761904762),
 'AUPR': np.float64(0.9336877361585658),
 'Accuracy': 0.7108433734939759,
 'Precision': 0.975609756097561,
 'Recall': 0.6349206349206349,
 'F1-Score': 0.7692307692307693,
 'Threshold': np.float32(0.084059946),
 'Normal_Score_Mean': np.float32(0.08314659),
 'Normal_Score_Std': np.float32(0.00062507926),
 'Anomaly_Score_Mean': np.float32(0.08504777)}